In [ ]:
pip install ortools

In [3]:
from ortools.sat.python import cp_model

In [71]:
model = cp_model.CpModel()

In [72]:
# variables

shifts = {'Morning': 'M', 'Afternoon': 'A', 'Evening': 'E'}
employees = ['Mark', 'Elon', 'David', 'Steve', 'Alan']
days = range(7) # 0: Monday etc.

schedule = {}

for i in employees:
  for j in days:
    for k in shifts.values():
      schedule[(i, j, k)] = model.NewBoolVar(f'{i}_{j}_{k}')

print(schedule)

{('Mark', 0, 'M'): Mark_0_M(0..1), ('Mark', 0, 'A'): Mark_0_A(0..1), ('Mark', 0, 'E'): Mark_0_E(0..1), ('Mark', 1, 'M'): Mark_1_M(0..1), ('Mark', 1, 'A'): Mark_1_A(0..1), ('Mark', 1, 'E'): Mark_1_E(0..1), ('Mark', 2, 'M'): Mark_2_M(0..1), ('Mark', 2, 'A'): Mark_2_A(0..1), ('Mark', 2, 'E'): Mark_2_E(0..1), ('Mark', 3, 'M'): Mark_3_M(0..1), ('Mark', 3, 'A'): Mark_3_A(0..1), ('Mark', 3, 'E'): Mark_3_E(0..1), ('Mark', 4, 'M'): Mark_4_M(0..1), ('Mark', 4, 'A'): Mark_4_A(0..1), ('Mark', 4, 'E'): Mark_4_E(0..1), ('Mark', 5, 'M'): Mark_5_M(0..1), ('Mark', 5, 'A'): Mark_5_A(0..1), ('Mark', 5, 'E'): Mark_5_E(0..1), ('Mark', 6, 'M'): Mark_6_M(0..1), ('Mark', 6, 'A'): Mark_6_A(0..1), ('Mark', 6, 'E'): Mark_6_E(0..1), ('Elon', 0, 'M'): Elon_0_M(0..1), ('Elon', 0, 'A'): Elon_0_A(0..1), ('Elon', 0, 'E'): Elon_0_E(0..1), ('Elon', 1, 'M'): Elon_1_M(0..1), ('Elon', 1, 'A'): Elon_1_A(0..1), ('Elon', 1, 'E'): Elon_1_E(0..1), ('Elon', 2, 'M'): Elon_2_M(0..1), ('Elon', 2, 'A'): Elon_2_A(0..1), ('Elon', 2, '

In [31]:
schedule

{('Mark', 0, 'M'): Mark_0_M(0..1),
 ('Mark', 0, 'A'): Mark_0_A(0..1),
 ('Mark', 0, 'E'): Mark_0_E(0..1),
 ('Mark', 1, 'M'): Mark_1_M(0..1),
 ('Mark', 1, 'A'): Mark_1_A(0..1),
 ('Mark', 1, 'E'): Mark_1_E(0..1),
 ('Mark', 2, 'M'): Mark_2_M(0..1),
 ('Mark', 2, 'A'): Mark_2_A(0..1),
 ('Mark', 2, 'E'): Mark_2_E(0..1),
 ('Mark', 3, 'M'): Mark_3_M(0..1),
 ('Mark', 3, 'A'): Mark_3_A(0..1),
 ('Mark', 3, 'E'): Mark_3_E(0..1),
 ('Mark', 4, 'M'): Mark_4_M(0..1),
 ('Mark', 4, 'A'): Mark_4_A(0..1),
 ('Mark', 4, 'E'): Mark_4_E(0..1),
 ('Mark', 5, 'M'): Mark_5_M(0..1),
 ('Mark', 5, 'A'): Mark_5_A(0..1),
 ('Mark', 5, 'E'): Mark_5_E(0..1),
 ('Mark', 6, 'M'): Mark_6_M(0..1),
 ('Mark', 6, 'A'): Mark_6_A(0..1),
 ('Mark', 6, 'E'): Mark_6_E(0..1),
 ('Elon', 0, 'M'): Elon_0_M(0..1),
 ('Elon', 0, 'A'): Elon_0_A(0..1),
 ('Elon', 0, 'E'): Elon_0_E(0..1),
 ('Elon', 1, 'M'): Elon_1_M(0..1),
 ('Elon', 1, 'A'): Elon_1_A(0..1),
 ('Elon', 1, 'E'): Elon_1_E(0..1),
 ('Elon', 2, 'M'): Elon_2_M(0..1),
 ('Elon', 2, 'A'): E

In [73]:
# Constraints

# 5 shifts per week
for i in employees:
  model.Add(sum(schedule[(i, day, shift)] for day in days for shift in shifts.values())==5)

# no more than 2 consecutive shifts
for i in employees:
  for j in range(5):
    for k in shifts.values():
      model.Add(sum(schedule[(i, z, k)] for z in range(j, j+3))<=2)

# Elon and Mark shouldn't work together
for i in days:
  for j in shifts.values():
    model.Add(schedule[('Mark', i, j)] + schedule[('Elon', i, j)] <=1)

# David and Steve shouldn't work together
for i in days:
  for j in shifts.values():
    model.Add(schedule[('David', i, j)] + schedule[('Steve', i, j)] <=1)

# Elon and Alan shouldn't work together
for i in days:
  for j in shifts.values():
    model.Add(schedule[('Elon', i, j)] + schedule[('Alan', i, j)] <=1)


# Mark and Steve shouldn't work together
for i in days:
  for j in shifts.values():
    model.Add(schedule[('Mark', i, j)] + schedule[('Steve', i, j)] <=1)

In [74]:
# solver func
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [75]:
# solving the problem

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution:')
  for i in days:
    print(f'Day {i}: ')
    for j in employees:
      for k in shifts.values():
        if solver.Value(schedule[(j, i, k)]):
          print(f'{j}: {k}')
  print()

else:
  print('No solution found.')


Solution:
Day 0: 
Steve: M
Steve: A
Steve: E
Alan: M
Alan: A
Alan: E
Day 1: 
Elon: E
David: E
Steve: M
Steve: A
Alan: M
Alan: A
Day 2: 
Elon: M
Elon: A
Elon: E
David: M
David: A
David: E
Day 3: 
Mark: A
Mark: E
Elon: M
David: M
Day 4: 
Mark: M
Mark: A
Mark: E
Day 5: 
Day 6: 

